Описание задания:

Что делать

Построить модель логистической регрессии, которая предсказывает уровень дохода.

При возможности попробуйте улучшить точность предсказаний (метод score) с помощью перебора признаков.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns; sns.set()

In [2]:
data = pd.read_csv('adult.csv')

In [3]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


Пустых значений в наших данных нет. Imputer нет необходимости использовать.

In [4]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [5]:
y = data.income
X = data.drop('income', axis=1)


Категориальный признак native-country с большой вариацией значений, из которых большинство Unites-States, заменю на 1  для Unites-States, 0 - для остальных.
Аналогичное кодирование делаю для содержимого gender

In [6]:
X['native-country'].value_counts().head()

United-States    43832
Mexico             951
?                  857
Philippines        295
Germany            206
Name: native-country, dtype: int64

In [7]:
X['native-country'] = X['native-country'].apply(lambda x: int(x == 'United-States'))
X['gender'] = X['gender'].apply(lambda x: int(x == 'male'))
X['native-country'].value_counts()

1    43832
0     5010
Name: native-country, dtype: int64

Далее по всем категориальным признакам провожу dummies кодирование.
Потом стандартизация всех получившихся признаков

In [8]:
cat_feat = list(X.dtypes[data.dtypes == 'object'].index)
num_feat = list(X.dtypes[data.dtypes == 'int64'].index)
X = pd.get_dummies(X[cat_feat], columns=cat_feat)

In [9]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y = pd.Series(data = le_y.fit_transform(y))

In [10]:
X.shape , y.shape

((48842, 61), (48842,))

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)

Делаю стратифицированное разбиение на train и test выборки

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)


In [13]:
X_train.head()

,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,...,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_0,native-country_0,native-country_1
34342,-0.246558,-0.173795,-0.26194,-0.01431,0.663711,-0.189609,-0.293019,-0.205606,-0.02074,-0.171088,...,-0.342391,-0.223869,-0.098572,-0.179161,-0.325728,-0.091554,0.411743,0.0,-0.338083,0.338083
18559,-0.246558,-0.173795,-0.26194,-0.01431,0.663711,-0.189609,-0.293019,-0.205606,-0.02074,5.844949,...,-0.342391,-0.223869,-0.098572,-0.179161,-0.325728,-0.091554,0.411743,0.0,-0.338083,0.338083
12477,-0.246558,-0.173795,-0.26194,-0.01431,0.663711,-0.189609,-0.293019,-0.205606,-0.02074,-0.171088,...,-0.342391,-0.223869,-0.098572,-0.179161,-0.325728,-0.091554,0.411743,0.0,2.957854,-2.957854
560,-0.246558,-0.173795,-0.26194,-0.01431,0.663711,-0.189609,-0.293019,-0.205606,-0.02074,-0.171088,...,2.920641,-0.223869,-0.098572,-0.179161,3.070047,-0.091554,-2.428701,0.0,-0.338083,0.338083
3427,-0.246558,-0.173795,-0.26194,-0.01431,0.663711,-0.189609,-0.293019,-0.205606,-0.02074,-0.171088,...,-0.342391,-0.223869,-0.098572,-0.179161,-0.325728,-0.091554,0.411743,0.0,-0.338083,0.338083


Логистическая регрессия с параметрами по умолчанию. L2-регуляризация, С=1

In [14]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

In [15]:
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
print('Правильность на обучающем наборе {:.4f}'.format(log_reg.score(X_train, y_train)))
print('Правильность на тестовом наборе {:.4f}'.format(log_reg.score(X_test, y_test)))


Правильность на обучающем наборе 0.8324
Правильность на тестовом наборе 0.8327


По полученнной точности на train и test видно, что модель недоучена. Попробуем улучшить 0.8327 подбором гиперпараметров.

In [17]:
for C in [0.001, 0.01, 0.1, 1, 10]:
    log_reg = LogisticRegression(penalty='l2',C=C)
    log_reg.fit(X_train, y_train)
    print('C = {}'.format(C))
    print('Правильность на обучающем наборе {:.4f}'.format(log_reg.score(X_train, y_train)))
    print('Правильность на тестовом наборе {:.4f}'.format(log_reg.score(X_test, y_test)))


C = 0.001
Правильность на обучающем наборе 0.8327
Правильность на тестовом наборе 0.8322
C = 0.01
Правильность на обучающем наборе 0.8324
Правильность на тестовом наборе 0.8326
C = 0.1
Правильность на обучающем наборе 0.8325
Правильность на тестовом наборе 0.8327
C = 1
Правильность на обучающем наборе 0.8324
Правильность на тестовом наборе 0.8327
C = 10
Правильность на обучающем наборе 0.8324
Правильность на тестовом наборе 0.8327


L2 регуляризация с перебором параметра C не улучашет точность 0.8327

In [18]:
for C in [0.001, 0.01, 0.1, 1, 10]:
    log_reg = LogisticRegression(penalty='l1',C=C)
    log_reg.fit(X_train, y_train)
    print('C = {}'.format(C))
    print('Правильность на обучающем наборе {:.4f}'.format(log_reg.score(X_train, y_train)))
    print('Правильность на тестовом наборе {:.4f}'.format(log_reg.score(X_test, y_test)))
    print('Количество используемых признаков {}'.format(np.sum(log_reg.coef_ != 0)))

C = 0.001
Правильность на обучающем наборе 0.8182
Правильность на тестовом наборе 0.8172
Количество используемых признаков 10
C = 0.01
Правильность на обучающем наборе 0.8316
Правильность на тестовом наборе 0.8315
Количество используемых признаков 43
C = 0.1
Правильность на обучающем наборе 0.8325
Правильность на тестовом наборе 0.8332
Количество используемых признаков 52
C = 1
Правильность на обучающем наборе 0.8324
Правильность на тестовом наборе 0.8327
Количество используемых признаков 55
C = 10
Правильность на обучающем наборе 0.8324
Правильность на тестовом наборе 0.8327
Количество используемых признаков 60


L1 регуляризаци с подбором параметра С позволяет немного повысить точность до 0.8332.

Тем не менее, судя по точности на обучающем наборе модель все равно недоучивается. Видимо, для слишком простая для данного набора данных. 
Повысить сложность модели можно было бы с помощью полиномиальных признаков, но не уверен, что для это применимо для логистической регрессии )


In [19]:
log_reg = LogisticRegression(penalty='l1',C=0.1)
log_reg.fit(X_train, y_train)
print('Правильность на обучающем наборе {:.4f}'.format(log_reg.score(X_train, y_train)))
print('Правильность на тестовом наборе {:.4f}'.format(log_reg.score(X_test, y_test)))
print('Количество используемых признаков {}'.format(np.sum(log_reg.coef_ != 0)))

Правильность на обучающем наборе 0.8325
Правильность на тестовом наборе 0.8332
Количество используемых признаков 52
